In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import bqplot as bq

In [ ]:
#read clean data and create dataframe
df = pd.read_csv("eda_final.csv")
df

In [ ]:
#make sure systolic blood pressure is an integer
df["systolic"].astype(int)

In [ ]:
#create function to categorize blood pressure
def blood_pressure_cat(systolic):
    if systolic >= 180:
        return "Crisis"
    elif (140 <= systolic < 180):
        return "HBP_stage2"
    elif (130 <= systolic < 140):
        return "HBP_stage1"
    elif (120 <= systolic < 130):
        return "Elevated"
    else:
        return "Normal"

In [ ]:
#run category function on dataframe
df['HTN_stage'] = df['systolic'].apply(lambda x: blood_pressure_cat(x))

In [ ]:
#create function to categorize age
def age_category(age):
    if age < 40:
        return "30-39"
    elif (50 > age >= 40):
        return "40-49"
    elif (60 > age >= 50):
        return "50-59"
    else:
        return "60-69"
#run category function on dataframe
df['age_range_y'] = df['age'].apply(lambda x: age_category(x))

In [ ]:
#creating handlers for data attibutes
gender_drop= widgets.Dropdown(options=[("Yes",1),
                                       ("No",0)],description="Include gender:",
                                         style= {'description_width': 'initial'})
run_button = widgets.Button(description="Show statistics",
                           button_style='success')

In [ ]:
#function to show first plot
def show_plot_1(b=None):
    gen=gender_drop.value
    plt.figure(figsize= (6,6))
    if gen == 0:
        ax =sns.countplot(y= 'HTN_stage', hue='gender', data= df, 
                          order=["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Systolic Blood Pressure Stages By Gender')
        plt.legend(["female", "male"])
    else:
        ax =sns.countplot(y= 'HTN_stage', data= df, 
                          order=["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Systolic Blood Pressure Stages Overview')   
    plt.ylabel('HTN stage')
    total = len(df['age_range'])
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))

#function to show second plot
def show_plot_2(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'age_range', hue= 'HTN_stage', data= df, order=["30-39","40-49","50-59","60-69"])
    plt.title('Systolic Blood Pressure Categories For Age Ranges', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Age ranges')
    plt.ylabel('Number of Patients')
    plt.legend(["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"])

#function to show third plot    
def show_plot_3(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'HTN_stage', hue= 'cardio', 
                  data= df, order=["Normal","Elevated","HBP_stage1", "HBP_stage2", "Crisis"],
                 palette="Set1")
    plt.title('Systolic Blood Pressure Categories With Cardio Disease Coexistance', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Systolic Blood Pressure Categories')
    plt.ylabel('Number of Patients')
    plt.legend(["cardio disease", "no cardio disease"])

In [ ]:
#tabs
@run_button.on_click
def plot_on_click(b):
    out.clear_output(wait=True)
    with out:
        show_plot_1()
        show_plot_2()
        show_plot_3()
        plt.show()

out=widgets.Output()
widget_box=widgets.VBox([gender_drop,run_button])

tab0 = widgets.Text('Age')
tab1 = widgets.Text('BMI')
tab2 = widgets.Text('Blood Pressure')
tab3 = widgets.HBox([widget_box,out])
tab4 = widgets.Text('Diastolic BP')

tab_contents = ['Age', 'BMI', 'Blood Pressure', 'Systolic Blood Pressure', 'Diastolic Blood Pressure']
children = [tab0, tab1, tab2, tab3, tab4]
tab = widgets.Tab()
tab.children = children
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
tab